In [ ]:
!pip install -qq peft wandb trl transformers datasets
!pip install -qq -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, Dataset
from huggingface_hub import login
from trl import SFTConfig, SFTTrainer, apply_chat_template
import wandb
import os
from peft import LoraConfig, get_peft_model

In [ ]:
login('')
wandb.login(key='')
wandb.init(project="lora-colab-reasoning")

In [ ]:
dtype = torch.bfloat16
max_length = 10000
num_epochs = 1
learning_rate = 2e-5
num_proc = 8
model_name = "wanhin/lora_cad_l3_merge"
output_dir = f"{model_name.split('/')[-1]}_lora_reasoning"
print(f"Training model: {output_dir}")


Training model: lora_cad_l3_merge_lora_reasoning


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    model_max_length=max_length,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
    device_map="cuda",
    use_cache=False
)

# Configure LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained("wanhin/lora_cad_2e",torch_dtype=torch.float16)

# model = model.to("cuda")

# model.enable_adapter_layers()

# Enable training mode explicitly
model.train()

model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

In [ ]:

raw_train_dataset = load_dataset("wanhin/cad_multi_reason_10k", split="train")
raw_val_dataset = load_dataset("wanhin/cad_multi_reason_10k", split="val")

raw_train_dataset = raw_train_dataset.to_pandas()
raw_train_dataset = [{"prompt": str(item["prompt"]), "completion": str(item["completion"])} for _, item in raw_train_dataset[150000:200000].iterrows()]

train_dataset_dict = {
    "prompt": [[{"role": "user", "content": item["prompt"]}] for item in raw_train_dataset],
    "completion": [[{"role": "assistant", "content": item["completion"]}] for item in raw_train_dataset]
}

val_dataset_dict = {
    "prompt": [[{"role": "user", "content": item["prompt"]}] for item in raw_val_dataset],
    "completion": [[{"role": "assistant", "content": item["completion"]}] for item in raw_val_dataset]
}

train_dataset = Dataset.from_dict(train_dataset_dict)
train_dataset = train_dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer}, num_proc=num_proc)

val_dataset = Dataset.from_dict(val_dataset_dict)
val_dataset = val_dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer}, num_proc=num_proc)

training_args = SFTConfig(
    dataset_num_proc = 1,
    max_length = max_length,
    completion_only_loss = True,
    output_dir=f"./train_results/{output_dir}",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=256,
    learning_rate=learning_rate,
    lr_scheduler_type = "constant",
    bf16=True,
    save_strategy="epoch",
    save_total_limit=2,
    save_safetensors=True,
    logging_steps=2,
    eval_strategy="steps",
    eval_steps=20,
    remove_unused_columns=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim="adamw_8bit",
    warmup_steps=0,
    report_to="wandb",
    save_only_model=True,
    max_grad_norm=1.0
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
try:
    trainer.train()
except Exception as e:
    print("Training failed")
    print(f"Error: {e}")

if wandb.run is not None:
    wandb.finish()
    print("wandb finished")

In [ ]:
repo_name = "wanhin/lora_cad_2e_l4"

# Push lên HF Hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

In [ ]:
# model.load_adapter("wanhin/lora_cad_2e", adapter_name="existing_lora")
# model.set_adapter("existing_lora")
# print("Successfully loaded existing LoRA weights!")
# from peft import AutoPeftModelForCausalLM

# model1 = AutoPeftModelForCausalLM.from_pretrained("wanhin/lora_cad_2e")

In [ ]:
model_merge = model.merge_and_unload()

repo_name = "wanhin/lora_cad_l4_merge"

# Push lên HF Hub
model_merge.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)